# **Model and Evaluation**

## Objectives

* Create Train and Test set and Machine Learning Pipeline to answer the second business requirement which is:
    * The client is interested in predicting the house sale price from her four inherited houses and any other house in Ames, Iowa.

## Inputs

* The clean and feature engineered datasets are located at: outputs/dataset/collection

## Outputs

* Train set (features and target)* 
Test set (features and target
* Modeling Pipeline. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/Fabrizio-Project-Five/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/Fabrizio-Project-Five'

Before starting our model notebook we will create a cell where we will place all of our imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
# Feature Engineering
from feature_engine.selection import SmartCorrelatedSelection
# Feature Scaling
from sklearn.preprocessing import StandardScaler
# Feature Selection
from sklearn.feature_selection import SelectFromModel
# Import GridSearchCV for model score evaluation
from sklearn.model_selection import GridSearchCV
# ML algorithms
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
# metrics for model evaluation
from sklearn.metrics import r2_score

# Splitting the Dataset

In this section we will split the dataset in Train and Test set in order to prepare it for model training. The test set will be 20% of the whole dataset just to make sure we are adherent to ML initial best practices.

In [ ]:
df_fe = pd.read_csv('outputs/dataset/collection/feature_engineered_dataset.csv')
df_fe.info()

In [ ]:
X = df_fe.drop('SalePrice', axis=1)
y = df_fe['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=9)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)

As we can see from the result of the .shape attribute called on the various test and train datasets we have achieved our goal. Now we can move on to the ML Pipeline itself.

---

# Regression Pipeline

In this section we will study which regression algorithm is the best one fo our ML business case. The results from the algorithm will also provide us with information that we can use to determine if additional hyperparameter optimization will be needed.

In this cell I have imported code from the Churnometer Walkthrough Project. These two dictionaries list first the type of regression
model we want to try for our ML business case and second a dictionary which will run a search for the default hyperparameters for each model

In [ ]:
# Code from the Churnometer Walkthrough Project

models_quick_search = {
    'LinearRegression': LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
}

In the next cell I will import a custom class from the Churnometer Project. Specifically this class will fit all the models listed in the in the previous dictionary and will apply the default hyperparameters (again, as listed in the previous dictionary). The class will also return a DataFrame with the scores for the regression models. This will allow us to select the best model for our Business Case.

In [ ]:
# Code from Churnometer Walkthrough Project
class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = regression_pipeline(self.models[key])

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches

In the next cell we will instantiate the Pipeline. The estimators will be described as well.

In [ ]:
def regression_pipeline(model):
    pipeline_base = Pipeline([
        # The SmartCorrelatedSelection is set with a threshold of 0.5 given our previous correlation study
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None,
         method="spearman", threshold=0.5, selection_method="variance")),
        # The feature scaling transformer has been applied because there were features with different units in the clean dataset 
        ("feature_scaling", StandardScaler()),
        # This transformer will select the best feature for prediction
        ("feature_selection",  SelectFromModel(model)),

        ("model", model),

    ])

    return pipeline_base    

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

In the above cell we can see that the best regressor for our business case is the 'Random Forest Regressor'. The mean score for 'R2' is already above the 0.75 value that we set as our goal. Nonetheless we can still conduct a more in-depth hyperparameter search to look for a fine tuned model that could give us even a more precise outcome.

In [ ]:
models_quick_search = {
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
}

params_quick_search = {
    'RandomForestRegressor': {
        'model__n_estimators': [10, 50, 100, 200, 300],
        'model__max_depth': [None, 10, 20, 30],
        'model__min_samples_split': [2, 5, 10, 20]
    }
}

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=10)

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary.query('mean_score > 0.80')

As we can see from our previous cells, even after 640 fits of our pipeline the best hyperparameters were found on the very first fit. 

* Model max depth: The best value for this hyperparameter is 10. That means that the algorithm ran until the trees in the forest reached a depth of 10
* Minimum sample split: The best value for this is 5. This means that 5 was the minimum number of samples required to split and internal node.
* Number of Estimators: The best value for this is 200. This means that we had 200 trees in our forest.

Now if we run the next cell we have our best regression pipeline, including the hyperparameters that gave us the best R2 score.

In [ ]:
best_model = grid_search_summary.iloc[0,0]
grid_search_pipelines[best_model].best_params_
regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
regressor_pipeline

After having found our best pipeline we need to evaluate the performance using the metrics we have decided in our business case assesment. 

In [ ]:
prediction = regressor_pipeline.predict(X_train)
regression_r2_metric = r2_score(y_train, prediction)
test_set_prediction = regressor_pipeline.predict(X_test)
r2_metric_test_set = r2_score(y_test, test_set_prediction)
print('r2 metric on train set: ', regression_r2_metric, '\n'
      'r2 metric on test set: ', r2_metric_test_set)
    

Our r2 score for the test set is too low while on the train set is too high. A classic case of overfitting. Our best move now is to tweak our hyperparameters to see if the pipeline will give us a test set r2 score that we can accept.

In [ ]:
def regression_pipeline(model):
    pipeline_base = Pipeline([
        # The SmartCorrelatedSelection is set with a threshold of 0.5 given our previous correlation study
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None,
         method="spearman", threshold=0.7, selection_method="variance")),
        # The feature scaling transformer has been applied because there were features with different units in the clean dataset 
        ("feature_scaling", StandardScaler()),
        # This transformer will select the best feature for prediction
        ("feature_selection",  SelectFromModel(model)),

        ("model", model),

    ])

    return pipeline_base 

In [ ]:
models_quick_search = {
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
}

params_quick_search = {
    'RandomForestRegressor': {
        'model__n_estimators': [10, 50, 100],
        'model__max_depth': [10, 20, 30],
        'model__min_samples_split': [2, 5, 10, 20]
    }
}

# For reference these below were our previous hyperparameters
"""
params_quick_search = {
    'RandomForestRegressor': {
        'model__n_estimators': [10, 50, 100, 200, 300],
        'model__max_depth': [None, 10, 20, 30],
        'model__min_samples_split': [2, 5, 10, 20]
    }
}
"""

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=20)

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

Now that we have a new best set of hyperparameters we can run our r2 score evaluation again.

In [ ]:
best_model = grid_search_summary.iloc[0,0]
grid_search_pipelines[best_model].best_params_
regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
regressor_pipeline

In [69]:
prediction = regressor_pipeline.predict(X_train)
regression_r2_metric = r2_score(y_train, prediction)
test_set_prediction = regressor_pipeline.predict(X_test)
r2_metric_test_set = r2_score(y_test, test_set_prediction)
print('r2 metric on train set: ', regression_r2_metric, '\n'
      'r2 metric on test set: ', r2_metric_test_set)

r2 metric on train set:  0.9397535641227075 
r2 metric on test set:  0.7644696609970755


/home/codeany/.local/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/home/codeany/.local/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/home/codeany/.local/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/home/codeany/.local/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passi

---

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
